In [6]:

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import caffe
import sys
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
from util import get_cla
from os.path import join, basename
import os
from get_accuracy import get_accuracy
from util import get_cla
import numpy as np

In [7]:
def do_train(solver, cl_args, its_per_epoch):
    num_epochs = cl_args["num_epochs"]
    losses = {}
    accuracies = []
    accuracies_loose = []
    for ep in range(num_epochs):
        losses = get_one_epoch_loss(ep, losses, its_per_epoch, solver)
        ep_losses = [losses[str(epoch) + "_mean"]["final_loss"] for epoch in range(ep + 1)]
        save_plot(ep_losses,cl_args, typ="loss")
        
        
        
        acc = get_accuracy(solver.net, 
                                  iterations=its_per_epoch,
                                  iou_thresh=0.5)
        

        accuracies.append(acc)
        save_plot(accuracies,cl_args, typ="acc")
        acc_loose = get_accuracy(solver.net, 
                          iterations=its_per_epoch,
                          iou_thresh=0.1)
        accuracies_loose.append(acc_loose)
        save_plot(accuracies_loose,cl_args, typ="acc_loose")
        
        


def get_one_epoch_loss(ep, losses, num_its, solver):
    loss_keys = ['L_cls', 'L_no_obj',
                             'L_obj',
                             'L_xy',
                             'L_wh',
                             'L_rec',
                             'final_loss']
    coeffs =  [1.0,1.0, 10.0, 1.0, 1.0, 3.39084201389e-08,1.0]
    loss_to_coeff_map = dict(zip(loss_keys, coeffs))
    losses[ep] = {k:[] for k in loss_keys}
    losses[str(ep) + "_mean"] = {k:0 for k in loss_keys}
    
    for it in range(num_its):
        solver.step(1)

        losses = write_iteration_losses(losses, solver,loss_keys,ep,it, loss_to_coeff_map)

    losses = write_epoch_losses(losses, loss_keys,ep, loss_to_coeff_map)
    return losses
    

def save_plot(arr, cl_args, typ="loss"):
    suffix = "%s_curve_%s_resumed.jpg"%(typ,basename(cl_args["solver_path"]))
    plt.figure(1)
    plt.clf()
    plt.plot(arr)
    plt.title(typ + " curve")
    plt.ylabel(typ)
    plt.xlabel("epochs")
    plt.show()
    plt.savefig(join(cl_args["save_dir"],"tr_" + suffix))
    

def write_epoch_losses(losses, loss_keys,ep, loss_to_coeff_map):
    losses[str(ep) + "_mean"] = {k:np.mean(losses[ep][k]) for k in loss_keys}
    for k in loss_keys:
        loss = losses[str(ep) + "_mean"][k]
        sys.stderr.write("\n Epoch %i: Final %s Loss = %6.3f   After coefficient: %6.3f\n" % (ep, k, loss, loss*loss_to_coeff_map[k]))
        sys.stderr.flush()
    return losses

def write_iteration_losses(losses, solver,loss_keys,ep,it, loss_to_coeff_map):
    for k in loss_keys:
        loss = np.float32(solver.net.blobs[k].data)
        
        losses[ep][k].append(loss)
        sys.stderr.write("\n Loss at iteration %i for %s is %8.4f    After coefficent %8.4f " % (it,k,loss,loss*loss_to_coeff_map[k]))
        sys.stderr.flush()
    return losses

In [9]:
if __name__ == "__main__":
    os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
    base_path = "/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/"
    
    cl_args = {"solver_path": join(base_path,"base_prototxt_files/solver_vanilla.prototxt"),"save_dir":join(base_path,"pycaffe_code", "plots"),
                                   "iou_thresh": 0.1, "num_epochs": 300, "solver_state_path": "/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/models/vanilla__iter_1000.solverstate"}
    
    its_per_epoch = 10
    
    cl_args = get_cla(cl_args)
    solver = caffe.SGDSolver(cl_args["solver_path"])
    if cl_args["solver_state_path"]:
        solver.restore(cl_args["solver_state_path"])
    do_train(solver, cl_args, its_per_epoch)
    
    

KeyboardInterrupt: 

In [3]:
#! jupyter nbconvert --to script train.ipynb

[NbConvertApp] Converting notebook train.ipynb to script
[NbConvertApp] Writing 4039 bytes to train.py
